In [ ]:
import numpy as np
import pandas as pd


## DATA

In [ ]:

df = pd.read_csv('http://fimi.uantwerpen.be/data/retail.dat', sep='\sp', engine='python' , header=None)
data = []
for i in range(df.shape[0]):
  arr = []
  arr.append(i)
  x = [int(i) for i in df[0][i].split()]
  arr.append(x);
  data.append(arr)
#data

In [ ]:
# testing data
data = [
        ['T100',['I1','I2','I5']],
        ['T200',['I2','I4']],
        ['T300',['I2','I3']],
        ['T400',['I1','I2','I4']],
        ['T500',['I1','I3']],
        ['T600',['I2','I3']],
        ['T700',['I1','I3']],
        ['T800',['I1','I2','I3','I5']],
        ['T900',['I1','I2','I3']]
        ]

## initialisation

In [ ]:
C = {}
L = {}

num_trans = len(data)
min_support = 0.2
min_conf = 0.3
itemset_size = 1
discarded = {itemset_size: []}
init = []
for i in data:
    for q in i[1]:
        if(q not in init):
            init.append(q)
init = sorted(init)
C.update({itemset_size : [[f] for f in init]})
C

{1: [['I1'], ['I2'], ['I3'], ['I4'], ['I5']]}

## helper functions

In [ ]:
def count_occurences(itemset, data):
  count = 0
  for i in range(len(data)):
    if set(itemset).issubset(set(data[i][1])):
      count += 1
  return count

In [ ]:
def get_frequent(itemset, data, min_support, prev_discarded ):
  L = []
  supp_counts = []
  new_discarded = []
  num_trans = len(data)

  k = len(prev_discarded.keys())

  for s in range(len(itemset)):
    discarded_before = False
    if k > 0:
      for it in prev_discarded[k]:
        if set(it).issubset(set(itemset[s])):
          discarded_before = True
          break
    if not discarded_before:
      count = count_occurences(itemset[s], data)
      if count/num_trans >= min_support:
        L.append(itemset[s])
        supp_counts.append(count)
      else:
        new_discarded.append(itemset[s])

  return L, supp_counts, new_discarded

In [ ]:
def print_table(L, supp_count):
  print("Itemsets | Support")
  for k in range(len(L)):
    print("{} : {}".format(L[k], supp_count[k]))
  print("\n\n")

## Creating L1

In [ ]:
supp_count_L = {}
f, sup, new_discarded = get_frequent(C[itemset_size], data, min_support, discarded)
discarded.update({itemset_size : new_discarded})
L.update({itemset_size : f})
supp_count_L.update({itemset_size : sup})

In [ ]:
print_table(L[1], supp_count_L[1])

Itemsets | Support
['I1'] : 6
['I2'] : 7
['I3'] : 6
['I4'] : 2
['I5'] : 2





## Using the apriori property

In [ ]:
def join_two(it1, it2, init):
  it1.sort( key = lambda x: init.index(x))
  it2.sort( key = lambda x: init.index(x))

  for i in range(len(it1)-1):
    if it1[i] != it2[i]:
      return []

  if init.index(it1[-1]) < init.index(it2[-1]):
    return it1 + [it2[-1]] 
  
  return []

In [ ]:
def join_itemsets(itemsets, init):
  C = []
  for i in range(len(itemsets)):
    for j in range(i+1, len(itemsets)):
      it_out = join_two(itemsets[i], itemsets[j], init)
      if len(it_out) > 0:
        C.append(it_out)
  return C

In [ ]:
k = itemset_size + 1
convergence = False
while not convergence:
  C.update({k : join_itemsets(L[k-1], init)})
  print("Table C{}: \n".format(k))
  print_table(C[k], [count_occurences(it, data) for it in C[k]])
  f, sup, new_discarded = get_frequent(C[k], data, min_support, discarded)
  L.update({k : f})
  discarded.update({k : new_discarded})
  supp_count_L.update({k : sup})
  if len(L[k]) == 0:
    convergence = True
  else :
    print("Table L{}: \n".format(k))
    print_table(L[k], supp_count_L[k])
  k += 1  

Table C2: 

Itemsets | Support
['I1', 'I2'] : 4
['I1', 'I3'] : 4
['I1', 'I4'] : 1
['I1', 'I5'] : 2
['I2', 'I3'] : 4
['I2', 'I4'] : 2
['I2', 'I5'] : 2
['I3', 'I4'] : 0
['I3', 'I5'] : 1
['I4', 'I5'] : 0



Table L2: 

Itemsets | Support
['I1', 'I2'] : 4
['I1', 'I3'] : 4
['I1', 'I5'] : 2
['I2', 'I3'] : 4
['I2', 'I4'] : 2
['I2', 'I5'] : 2



Table C3: 

Itemsets | Support
['I1', 'I2', 'I3'] : 2
['I1', 'I2', 'I5'] : 2
['I1', 'I3', 'I5'] : 1
['I2', 'I3', 'I4'] : 0
['I2', 'I3', 'I5'] : 1
['I2', 'I4', 'I5'] : 0



Table L3: 

Itemsets | Support
['I1', 'I2', 'I3'] : 2
['I1', 'I2', 'I5'] : 2



Table C4: 

Itemsets | Support
['I1', 'I2', 'I3', 'I5'] : 1





## generating association rules

In [ ]:


from itertools import combinations,chain

def powerset(s):
  return list(chain.from_iterable(combinations(s,r) for r in range(1 , len(s)+1 )))

def write_rules(X, X_S, S, conf, sup_x, lift, num_trans):
  out_string = ""
  out_string += "Freq. Itemset: {}\n".format(X) 
  out_string += " Rule: {}  ->  {}\n".format(list(S), list(X_S))
  out_string += "    Conf: {0:2.3f}".format(conf)
  out_string += "    Supp: {0:2.3f}".format(sup_x/num_trans)
  out_string += "    Lift: {0:2.3f}\n".format(lift)
  return out_string


In [ ]:
assoc_rules = ""
for i in range(1, len(L)):
  for j in range(len(L[i])):
    s = powerset(set(L[i][j]))
    s.pop()
    for z in s:
      S = set(z)
      X = set(L[i][j])
      X_S = set(X-S)
      sup_x = count_occurences(X, data)
      sup_x_s = count_occurences(X_S, data)
      conf = sup_x/ count_occurences(S, data)
      lift = conf / (sup_x_s / num_trans)
      if conf >= min_conf and sup_x >= min_support:
        assoc_rules += write_rules(X, X_S, S, conf, sup_x, lift, num_trans)
print(assoc_rules)

Freq. Itemset: {'I1', 'I2'}
 Rule: ['I1']  ->  ['I2']
    Conf: 0.667    Supp: 0.444    Lift: 0.857
Freq. Itemset: {'I1', 'I2'}
 Rule: ['I2']  ->  ['I1']
    Conf: 0.571    Supp: 0.444    Lift: 0.857
Freq. Itemset: {'I3', 'I1'}
 Rule: ['I3']  ->  ['I1']
    Conf: 0.667    Supp: 0.444    Lift: 1.000
Freq. Itemset: {'I3', 'I1'}
 Rule: ['I1']  ->  ['I3']
    Conf: 0.667    Supp: 0.444    Lift: 1.000
Freq. Itemset: {'I5', 'I1'}
 Rule: ['I5']  ->  ['I1']
    Conf: 1.000    Supp: 0.222    Lift: 1.500
Freq. Itemset: {'I5', 'I1'}
 Rule: ['I1']  ->  ['I5']
    Conf: 0.333    Supp: 0.222    Lift: 1.500
Freq. Itemset: {'I3', 'I2'}
 Rule: ['I3']  ->  ['I2']
    Conf: 0.667    Supp: 0.444    Lift: 0.857
Freq. Itemset: {'I3', 'I2'}
 Rule: ['I2']  ->  ['I3']
    Conf: 0.571    Supp: 0.444    Lift: 0.857
Freq. Itemset: {'I4', 'I2'}
 Rule: ['I4']  ->  ['I2']
    Conf: 1.000    Supp: 0.222    Lift: 1.286
Freq. Itemset: {'I5', 'I2'}
 Rule: ['I5']  ->  ['I2']
    Conf: 1.000    Supp: 0.222    Lift: 1.286
